# Overview

In this project my aim is to develop my own linear regression function from scratch - the performance of which I will benchmark against SKlearn

My 'DIY' linear regression models will use self-built gradient descent functions. And I will judge the relative performance of my model by:

- It's MSE and R^2 relative to sklearn's
- Whether it arrives at the same coefficients
- Whether it completes it operations in a manageable timeframe


Focusing on applying the model to one dataset with one independent variable initially, I will improve the model as needed to optimize its performance. I will then apply the model to a further one-independent variable dataset, to test its range.

# Setting benchmarks - SK learn's performance

In this section, I will use a dataset from kaggle, which shows how Sales vary with TV marketing budgets.

I will use a cross-validation method to find the spread of:
- performances relative to MSE/R^2
- estimated coefficients
- durations to complete operations


I will later use these metrics as benchmarks against which to measure the performance of my DIY model

In [ ]:
import numpy as np
import pandas as pd

ads = pd.read_csv('tvmarketing.csv')
ads.head(10)

,TV,Sales
0,230.1,22.1
1,44.5,10.4
2,17.2,9.3
3,151.5,18.5
4,180.8,12.9
5,8.7,7.2
6,57.5,11.8
7,120.2,13.2
8,8.6,4.8
9,199.8,10.6


In [ ]:
ads.isna().mean()

TV       0.0
Sales    0.0
dtype: float64

In [ ]:
len(ads)

200

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import time
import math


In [ ]:
#don't want to use sklearn cross validation function, will make my own as it will be easier to get the coefficients out for each run

#what I want to produce is a table giving following information for each run:
#- number of the run
#- MSE
#- R^2
# - gradient coefficient
# - intercept coefficient
# - completion time




In [ ]:
#set up different test/train combinations, such that each observation is used 4 times to train, and only tested on once

test_1_index = np.arange(0,40)
test_2_index = np.arange(40,80)
test_3_index = np.arange(80,120)
test_4_index = np.arange(120,160)
test_5_index = np.arange(160,200)

def train_index(test_index):

  full_index_set = set(np.arange(0,200))
  test_index_set = set(test_index)

  train_index_set = full_index_set - test_index_set

  train_index_list = list(train_index_set)

  train_index_array = np.array(train_index_list)

  return train_index_array



train_1_index = train_index(test_1_index)
train_2_index = train_index(test_2_index)
train_3_index = train_index(test_3_index)
train_4_index = train_index(test_4_index)
train_5_index = train_index(test_5_index)






In [ ]:
test_1 = ads.iloc[test_1_index]
test_2 = ads.iloc[test_2_index]
test_3 = ads.iloc[test_3_index]
test_4 = ads.iloc[test_4_index]
test_5 = ads.iloc[test_5_index]

test_sets = [test_1, test_2, test_3, test_4, test_5]


train_1 = ads.iloc[train_1_index]
train_2 = ads.iloc[train_2_index]
train_3 = ads.iloc[train_3_index]
train_4 = ads.iloc[train_4_index]
train_5 = ads.iloc[train_5_index]

train_sets = [train_1, train_2, train_3, train_4, train_5]


In [ ]:
train_sets[0].TV

40     202.5
41     177.0
42     293.6
43     206.9
44      25.1
       ...  
195     38.2
196     94.2
197    177.0
198    283.6
199    232.1
Name: TV, Length: 160, dtype: float64

In [ ]:
storage = pd.DataFrame({'Run':[], 'MSE':[], 'R^2': [], 'a': [], 'b':[], 'completion_time':[]})

for i in range(0, 5):

  start = time.time()

  X_train = train_sets[i].TV.to_frame()
  y_train = train_sets[i].Sales

  X_test = test_sets[i].TV.to_frame()
  y_test = test_sets[i].Sales

  model = LinearRegression()
  model.fit(X_train,y_train)

  predictions = model.predict(X_test)

  a = model.coef_.item()
  b = model.intercept_.item()

  r2 = model.score(X_test, y_test)
  mse = mean_squared_error(y_test, predictions, squared = True)


  storage_row = pd.DataFrame({'Run':[i+1], 'MSE':[round(mse,3)], 'R^2': [round(r2,3)], 'a': [round(a,3)], 'b':[round(b,3)], 'completion_time':[time.time()- start]})

  storage = pd.concat([storage, storage_row], axis = 0)


storage = storage.reset_index()


storage_means = pd.DataFrame({'Run':['means'], 'MSE':[storage.MSE.mean()], 'R^2': [storage['R^2'].mean()], 'a': [storage.a.mean()], 'b':[storage.b.mean()], 'completion_time':[storage['completion_time'].mean()]})


storage_stds = pd.DataFrame({'Run':['stds'], 'MSE':[storage.MSE.std()], 'R^2': [storage['R^2'].std()], 'a': [storage.a.std()], 'b':[storage.b.std()], 'completion_time':[storage['completion_time'].std()]})


storage = pd.concat([storage, storage_means, storage_stds], axis = 0)


storage.drop(['index'], axis = 1, inplace = True)

storage.to_csv('sklearn_benchmarks_tv.csv')



In [ ]:
storage

,Run,MSE,R^2,a,b,completion_time
0,1.0,10.497000,0.594000,0.049000,6.870000,0.005248
1,2.0,9.200000,0.688000,0.045000,7.190000,0.004004
2,3.0,9.673000,0.569000,0.048000,6.887000,0.003427
3,4.0,10.474000,0.642000,0.047000,7.129000,0.003382
4,5.0,14.129000,0.471000,0.049000,7.069000,0.003467
0,means,10.794600,0.592800,0.047600,7.029000,0.003905
0,stds,1.943599,0.081986,0.001673,0.144019,0.000792


I will gather equivalent insights on my model's performance and compare to these sklearn benchmarks

# Building first version of my Linear Regression function

In this section, I will build the first version of my linear regression function.

I ultimately want a function that will:
- take in X_train, X_test, y_train, y_test, alpha (step scaling factor), n_iter(number of steps per run), n_trials (number of runs to find best parameters)
- give out MSE, R^2, a, b, completition time

In order to get to this point, I will build:
- A loss function, and gradient function for the loss function
- A function that estimates best a and b, given starting points
- A function that estimates best a and b, after starting at n_trials randomly selected starting points


Once I have build the overall function I will:
- cross validate the function using the five test/train splits from previous section
- vary alpha, n_iter, n_trials to see if any combinations yield satisfactory performance
- draw conclusions about how much I need to improve my function

Loss function, and its derivative:

- Sales_pred_as_vector --> Sp
- Actual_sales_as_vector --> S
- TV_marketing_as_vector --> T
- coefficient --> a
- intercept --> b
- 1d_vector_of_all_1s --> U (where u has same length as S,T, Sp)
- Sum_Squared_Error = SSE

Sp = aT + bU

SSE = (Sp-S).(Sp-S) [dot product]

SSE = a^2(T.T) + b^2(U.U) + S.S + 2ab(T.U) - 2a(T.S) - 2b(S.U)

delta_SSE = [2aT.T + 2bT.U - 2T.S], [2bU.U + 2aT.U - 2S.U]

delta_SSE = 2 * [[T.T, U.T, S.T],[T.U, U.U, S.U]] * [a,b,-1]_transposed


In [ ]:
def loss_function_uniV_LR (X, y, a, b):

  u = np.repeat(1, len(y))

  sse = ((a**2)*(np.dot(X,X))) + ((b**2)*(np.dot(u,u))) + np.dot(y,y) + ((2*a*b)*(np.dot(X,u))) - ((2*a)*(np.dot(X,y))) - ((2*b)*(np.dot(u,y)))

  return sse



In [ ]:
def loss_function_derivative_wrt_ab_UNIT(X,y,a,b):

  u = np.repeat(1, len(y))

  matrix = np.array([[np.dot(X,X), np.dot(u,X), np.dot(y,X)],
                    [np.dot(X,u), np.dot(u,u), np.dot(y,u)]])



  vector = np.array([a,b,-1])


  gradient_vector = 2*np.matmul(matrix,vector)

  length_grad_vector = ((gradient_vector[0]**2) + (gradient_vector[1]**2))**0.5

  gradient_vector_unit = gradient_vector/length_grad_vector





  return gradient_vector_unit



In [ ]:
def find_best_ab_for_given_start (X,y, a_start, b_start, alpha, n_iter):

  point = [a_start, b_start]

  grad = loss_function_derivative_wrt_ab_UNIT(X,y,a_start,b_start)

  movement = -1*alpha*grad

  for i in range(0, n_iter):

    point = point + movement

    grad = loss_function_derivative_wrt_ab_UNIT(X,y,point[0],point[1])

    movement = -1*alpha*grad


  a_est = point[0]
  b_est = point[1]


  return[a_est, b_est]

In [ ]:
def find_best_ab(X,y, alpha, n_iter, n_trials):

  a_starts = np.random.randint(-20,20, n_trials)
  b_starts = np.random.randint(-20,20, n_trials)

  a_estimates = []
  b_estimates = []

  for i in range(0, n_trials):

    point_estimates = find_best_ab_for_given_start (X,y, a_starts[i], b_starts[i], alpha, n_iter)

    a_estimates.append(point_estimates[0])
    b_estimates.append(point_estimates[1])


  SSR_values = []

  for i in range(0,n_trials):

    SSR_values.append(loss_function_uniV_LR (X, y, a_estimates[i], b_estimates[i]))

  SSR_values_min = min(SSR_values)

  min_index = SSR_values.index(SSR_values_min)

  a_min = a_estimates[min_index]
  b_min = b_estimates[min_index]

  return [a_min, b_min]










In [ ]:
def a_b_mse_r_squared (x_train, y_train, x_test, y_test, alpha, n_iter, n_trials):

  start = time.time()

  a = find_best_ab(x_train,y_train, alpha, n_iter, n_trials)[0]
  b = find_best_ab(x_train,y_train, alpha, n_iter, n_trials)[1]

  predictions = (a*x_test)+b

  errors = predictions - y_test

  MSE = (np.dot(errors, errors))/len(y_test)

  y_test_mean = np.mean(y_test)

  y_test_mean_vector = np.repeat(y_test_mean, len(y_test))

  errors_from_mean = y_test - y_test_mean_vector

  R_squared = 1 - ((np.dot(errors, errors))/(np.dot(errors_from_mean, errors_from_mean)))

  end = time.time()

  storage = pd.DataFrame({'MSE':[MSE], 'R^2': [R_squared], 'a': [a], 'b':[b], 'completion_time':[end - start]})

  return storage






In [ ]:
#use test and train sets developed in Setting Benchmark settings
#to test how my algorithm compares to SKLEARN



In [ ]:
#start with alpha, n_iter, n_trials = 0.2, 100000, 10

storage = pd.DataFrame({'MSE':[], 'R^2': [], 'a': [], 'b':[], 'completion_time':[]})

for i in range(0, 5):

  storage_row = a_b_mse_r_squared(train_sets[i].TV, train_sets[i].Sales , test_sets[i].TV, test_sets[i].Sales, 0.2, 100000, 10)

  storage = pd.concat([storage, storage_row], axis = 0)

storage

,MSE,R^2,a,b,completion_time
0,96.221915,-2.722681,0.110607,4.643138,214.930847
0,28.162784,0.043682,0.025457,7.210900,212.557981
0,135.630903,-5.046455,0.114988,6.912021,208.098088
0,17.496487,0.401664,0.068117,6.612299,209.766502
0,138.410600,-4.183146,-0.009567,5.173188,211.069405


In [ ]:
#alpha, n_iter, n_trials = 0.01, 100000, 20

storage = pd.DataFrame({'MSE':[], 'R^2': [], 'a': [], 'b':[], 'completion_time':[]})

for i in range(0, 5):

  storage_row = a_b_mse_r_squared(train_sets[i].TV, train_sets[i].Sales , test_sets[i].TV, test_sets[i].Sales, 0.01, 100000, 20)

  storage = pd.concat([storage, storage_row], axis = 0)

storage

,MSE,R^2,a,b,completion_time
0,11.717814,0.546655,0.039147,7.038163,421.586713
0,11.603841,0.605971,0.043152,6.690677,420.832090
0,11.458347,0.489184,0.044339,6.451273,423.311496
0,11.185780,0.617474,0.052657,7.262142,424.833284
0,15.912705,0.404107,0.051350,7.212117,428.222465


In [ ]:
#alpha, n_iter, n_trials = 0.01, 10000, 20

storage = pd.DataFrame({'MSE':[], 'R^2': [], 'a': [], 'b':[], 'completion_time':[]})

for i in range(0, 5):

  storage_row = a_b_mse_r_squared(train_sets[i].TV, train_sets[i].Sales , test_sets[i].TV, test_sets[i].Sales, 0.01, 10000, 20)

  storage = pd.concat([storage, storage_row], axis = 0)

storage

,MSE,R^2,a,b,completion_time
0,18.752466,0.274495,0.059161,7.785277,50.060354
0,9.452344,0.679029,0.040587,8.684005,77.898331
0,9.658057,0.569442,0.040936,8.704213,59.057460
0,11.104812,0.620243,0.040462,7.835406,43.856665
0,15.930587,0.403438,0.039831,5.545159,42.453114


In [ ]:
#the three tables above show that:

#alpha of 0.01 is by far preferable to alpha 0.2
#increasing n_iter from 10000 to 100000 does not give significant benefits in terms of accuracy
#relative to excess computational speed
#time to run for my model is significantly higher than sklearn

#I could do a gridsearch-style comparison of many hyperparameters(HPs) and rerun for same HPs to test effects of
#random starting point
#however, I am satisfied that my model needs fundamental improvements, and do not want to invest time confirming what I already know

# Building second version of my linear regression function - Scaling X

For my second version, the main change I am going to make is to scale the X feature.

As I will demonstrate, having the X feature (TV budget) unscaled has the effect of making the gradient of the loss function wrt a much larger than the gradient of the loss function wrt to b. This makes it harder to converge on best b, which is likely to also skew its estimates of the best a.

My approach will therefore be to scale TV budget data so that it has zero mean and unit variance.

In [ ]:
#demonstrating the effect of scaling on gradient of loss function values

#first I will create scaled versions of train_1 and test_1 for later use

train_1_TV_mean = train_1.TV.mean()
train_1_TV_sd = train_1.TV.std()

train_1['TV_scaled'] = (np.array(train_1.TV) - train_1_TV_mean)/train_1_TV_sd
test_1['TV_scaled'] = (np.array(test_1.TV ) - train_1_TV_mean)/train_1_TV_sd


<ipython-input-19-05103eee1138>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_1['TV_scaled'] = (np.array(train_1.TV) - train_1_TV_mean)/train_1_TV_sd
<ipython-input-19-05103eee1138>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_1['TV_scaled'] = (np.array(test_1.TV ) - train_1_TV_mean)/train_1_TV_sd


In [ ]:
#get gradients of loss function for combinations of values of a and b
#between -10 and 10, with interval of 2


storage_1 = pd.DataFrame({'a':[], 'b':[], 'loss_grad_wrt_a':[], 'loss_grad_wrt_b':[]})

for i in range(-10,10,2):

  for j in range(-10,10,2):

    gradient = loss_function_derivative_wrt_ab_UNIT(train_1.TV,train_1.Sales,i,j)


    storage_row_1 = pd.DataFrame({'a':[i], 'b':[j], 'loss_grad_wrt_a':[gradient[0]], 'loss_grad_wrt_b':[gradient[1]]})

    storage_1 = pd.concat([storage_1, storage_row_1], axis = 0)



pd.set_option('display.max_rows', None)

storage_1





,a,b,loss_grad_wrt_a,loss_grad_wrt_b
0,-10.0,-10.0,-0.999987,-0.005135
0,-10.0,-8.0,-0.999987,-0.005134
0,-10.0,-6.0,-0.999987,-0.005132
0,-10.0,-4.0,-0.999987,-0.005130
0,-10.0,-2.0,-0.999987,-0.005129
0,-10.0,0.0,-0.999987,-0.005127
0,-10.0,2.0,-0.999987,-0.005125
0,-10.0,4.0,-0.999987,-0.005124
0,-10.0,6.0,-0.999987,-0.005122
0,-10.0,8.0,-0.999987,-0.005120


In [ ]:
#here are the mean values of the gradients wrt a and b

[np.mean(np.abs(storage_1.loss_grad_wrt_a)), np.mean(np.abs(storage_1.loss_grad_wrt_b))]

[0.9999865335232464, 0.005184597307868766]

In [ ]:
#get equivalent information for scaled data
storage_2 = pd.DataFrame({'a':[], 'b':[], 'loss_grad_wrt_a':[], 'loss_grad_wrt_b':[]})

for i in range(-10,10,2):

  for j in range(-10,10,2):

    gradient = loss_function_derivative_wrt_ab_UNIT(train_1.TV_scaled,train_1.Sales,i,j)


    storage_row_2 = pd.DataFrame({'a':[i], 'b':[j], 'loss_grad_wrt_a':[gradient[0]], 'loss_grad_wrt_b':[gradient[1]]})

    storage_2 = pd.concat([storage_2, storage_row_2], axis = 0)




storage_2

,a,b,loss_grad_wrt_a,loss_grad_wrt_b
0,-10.0,-10.0,-0.504048,-0.863675
0,-10.0,-8.0,-0.536995,-0.843585
0,-10.0,-6.0,-0.573510,-0.819199
0,-10.0,-4.0,-0.613899,-0.789385
0,-10.0,-2.0,-0.658356,-0.752707
0,-10.0,0.0,-0.706834,-0.707380
0,-10.0,2.0,-0.758848,-0.651267
0,-10.0,4.0,-0.813193,-0.581994
0,-10.0,6.0,-0.867599,-0.497264
0,-10.0,8.0,-0.918466,-0.395499


In [ ]:
[np.mean(np.abs(storage_2.loss_grad_wrt_a)), np.mean(np.abs(storage_2.loss_grad_wrt_b))]

[0.3754863010781866, 0.8813883926524706]

In [ ]:
#Scaling TV data (the X variable) has had the effect of making the gradient with respect to a,
#and gradient with respect to b much more similar in magnitude. And given that
#the GD function uses a unit vector, this means that the function is taking steps
#that are sensitive to changes in both a and b, meaning it
#is more likely to find optimum values for both

In [ ]:
#scale the X values for each of the training sets

def create_scaled_X_column(df):

  X_us = np.array(df.TV)

  X_us_mean = np.mean(X_us)
  X_us_std = np.std(X_us)

  X_s = (X_us - X_us_mean)/X_us_std

  df_2 = df.copy(deep = True)

  df_2['TV_scaled'] = X_s

  return df_2

In [ ]:
train_1_S = create_scaled_X_column(train_1)

In [ ]:
train_2_S = create_scaled_X_column(train_2)
train_3_S = create_scaled_X_column(train_3)
train_4_S = create_scaled_X_column(train_4)
train_5_S = create_scaled_X_column(train_5)


test_1_S = create_scaled_X_column(test_1)
test_2_S = create_scaled_X_column(test_2)
test_3_S = create_scaled_X_column(test_3)
test_4_S = create_scaled_X_column(test_4)
test_5_S = create_scaled_X_column(test_5)

In [ ]:
train_sets_s = [train_1_S, train_2_S, train_3_S, train_4_S, train_5_S]
test_sets_s = [test_1_S, test_2_S, test_3_S, test_4_S, test_5_S]

In [ ]:
#set up SK learn benchmarks using scaled x data
#note that I will convert the coefficients for scaled X --> y, back to coefficients for x--y

#I have found conversion function by substituting X_scaled = (X - mean(X))/SD(X)
#into Yp = a_scaled*X_scaled + b_scaled*u
#to get Yp = (a_scaled/sd(X))*X + (b_s - (a_s*mean(X)/SD(X)))*U
#the coefficient infront of X will give a, that before U will give b

storage = pd.DataFrame({'Run':[], 'MSE':[], 'R^2': [], 'a': [], 'b':[], 'completion_time':[]})

for i in range(0, 5):

  start = time.time()

  X_train = train_sets_s[i].TV_scaled.to_frame()
  y_train = train_sets_s[i].Sales

  X_test = test_sets_s[i].TV_scaled.to_frame()
  y_test = test_sets_s[i].Sales

  model = LinearRegression()
  model.fit(X_train,y_train)

  predictions = model.predict(X_test)

  a_s = model.coef_.item()
  b_s = model.intercept_.item()

  a = a_s/np.std(train_sets_s[i].TV)
  b = b_s - ((a_s*np.mean(train_sets_s[i].TV))/np.std(train_sets_s[i].TV))

  r2 = model.score(X_test, y_test)
  mse = mean_squared_error(y_test, predictions, squared = True)


  storage_row = pd.DataFrame({'Run':[i+1], 'MSE':[round(mse,3)], 'R^2': [round(r2,3)], 'a': [round(a,3)], 'b':[round(b,3)], 'completion_time':[time.time()- start]})

  storage = pd.concat([storage, storage_row], axis = 0)


storage = storage.reset_index()


storage_means = pd.DataFrame({'Run':['means'], 'MSE':[storage.MSE.mean()], 'R^2': [storage['R^2'].mean()], 'a': [storage.a.mean()], 'b':[storage.b.mean()], 'completion_time':[storage['completion_time'].mean()]})


storage_stds = pd.DataFrame({'Run':['stds'], 'MSE':[storage.MSE.std()], 'R^2': [storage['R^2'].std()], 'a': [storage.a.std()], 'b':[storage.b.std()], 'completion_time':[storage['completion_time'].std()]})


storage = pd.concat([storage, storage_means, storage_stds], axis = 0)


storage.drop(['index'], axis = 1, inplace = True)

storage

,Run,MSE,R^2,a,b,completion_time
0,1.0,10.276000,0.602000,0.049000,6.870000,0.010138
1,2.0,8.784000,0.702000,0.045000,7.190000,0.005626
2,3.0,10.289000,0.541000,0.048000,6.887000,0.005265
3,4.0,12.046000,0.588000,0.047000,7.129000,0.005844
4,5.0,12.743000,0.523000,0.049000,7.069000,0.005888
0,means,10.827600,0.591200,0.047600,7.029000,0.006552
0,stds,1.575112,0.069955,0.001673,0.144019,0.002020


For my DIY function, I want to:

- take in X_train, y_train, x_test, y_test, alpha, n_iter, n_trials
- scale x_train and x_test using mean and std from x_train
- find best a_scaled, b_scaled (a_s, b_s for short)
- use a_scaled, b_scaled, and x_test_scaled to get y_pred
- compare y_pred and y to get MSE and R^2
- convert a_s, b_s to a, b
- report a,b, MSE, R^2, along with time taken for completion


In [ ]:
def a_b_mse_r_squared_WITH_SCALING (x_train, y_train, x_test, y_test, alpha, n_iter, n_trials):

  start = time.time()

  x_train_mean = np.mean(x_train)
  x_train_std = np.std(x_train)

  x_train_scaled = (x_train - x_train_mean)/x_train_std
  x_test_scaled = (x_test - x_train_mean)/x_train_std

  best_a_b = find_best_ab(x_train_scaled,y_train, alpha, n_iter, n_trials)

  a_s = best_a_b[0]
  b_s = best_a_b[1]

  predictions = (a_s*x_test_scaled) + b_s

  errors = predictions - y_test

  MSE = (np.dot(errors, errors))/len(y_test)

  y_test_mean = np.mean(y_test)

  y_test_mean_vector = np.repeat(y_test_mean, len(y_test))

  errors_from_mean = y_test - y_test_mean_vector

  R_squared = 1 - ((np.dot(errors, errors))/(np.dot(errors_from_mean, errors_from_mean)))

  a = a_s/x_train_std

  b = b_s - ((a_s*x_train_mean)/(x_train_std))

  end = time.time()

  storage = pd.DataFrame({'MSE':[MSE], 'R^2': [R_squared], 'a': [a], 'b':[b], 'completion_time':[end - start]})

  return storage

In [ ]:
storage = pd.DataFrame({'MSE':[], 'R^2': [], 'a': [], 'b':[], 'completion_time':[]})

for i in range(0, 5):

  storage_row = a_b_mse_r_squared_WITH_SCALING(train_sets[i].TV, train_sets[i].Sales , test_sets[i].TV, test_sets[i].Sales, 0.01, 10000, 20)

  storage = pd.concat([storage, storage_row], axis = 0)

storage

,MSE,R^2,a,b,completion_time
0,10.496685,0.593899,0.048771,6.868000,21.112621
0,9.199583,0.687612,0.045361,7.190023,20.261292
0,9.672395,0.568803,0.047929,6.887232,22.649185
0,10.474754,0.641790,0.046872,7.132108,21.557846
0,14.129280,0.470892,0.048940,7.068239,20.097969


In [ ]:
#values, other than time, similar to those found in sklearn benchmarks

In [ ]:
storage_means = pd.DataFrame({'Run':['means'], 'MSE':[storage.MSE.mean()], 'R^2': [storage['R^2'].mean()], 'a': [storage.a.mean()], 'b':[storage.b.mean()], 'completion_time':[storage['completion_time'].mean()]})


pd.concat ([storage_means, pd.DataFrame({'Run':['stds'], 'MSE':[storage.MSE.std()], 'R^2': [storage['R^2'].std()], 'a': [storage.a.std()], 'b':[storage.b.std()], 'completion_time':[storage['completion_time'].std()]})])

,Run,MSE,R^2,a,b,completion_time
0,means,10.794539,0.592599,0.047575,7.029120,21.135783
0,stds,1.943873,0.081896,0.001485,0.145016,1.038129


In [ ]:
#while time to complete is not comparable to sklearn, the time to run this algorithm is much more manageable
#and the accuracy is good

In [ ]:
#want to see how my model performs with different Hyperparameters, so will create a function that takes in HPs, performs a run using each of the
#five train/test data sets
#returns average and std of values for MSE, R2, a, b, time

In [ ]:
def DIY_LinReg_metrics_for_given_HP(alpha, n_iter, n_trials):

  storage = pd.DataFrame({'MSE':[], 'R^2': [], 'a': [], 'b':[], 'completion_time':[]})

  for i in range(0, 5):

    storage_row = a_b_mse_r_squared_WITH_SCALING(train_sets[i].TV, train_sets[i].Sales , test_sets[i].TV, test_sets[i].Sales, alpha, n_iter, n_trials)

    storage = pd.concat([storage, storage_row], axis = 0)

  return storage

In [ ]:
def DIY_LinReg_metrics_for_given_HP_MEANS(alpha, n_iter, n_trials):

  full_cv_df = DIY_LinReg_metrics_for_given_HP(alpha, n_iter, n_trials)

  storage = pd.DataFrame({'MSE':[full_cv_df['MSE'].mean()], 'R^2': [full_cv_df['R^2'].mean()],
                          'a': [full_cv_df['a'].mean()], 'b':[full_cv_df['b'].mean()],
                          'completion_time':[full_cv_df['completion_time'].mean()]})

  return storage



In [ ]:
#want to create a table that allows me to analyse effects of changing alpha, n_iter and n_trials

GridSearch_metrics = pd.DataFrame([(alpha, n_iter, n_trials) for alpha in [0.001, 0.01, 0.1, 1] for n_iter in [100,1000,10000] for n_trials in [5,10]],
                                  columns = ['alpha', 'n_iter', 'n_trials'])




In [ ]:
key_values = pd.DataFrame({'MSE': np.repeat(1, len(GridSearch_metrics)), 'R^2': np.repeat(1, len(GridSearch_metrics)),
                           'a': np.repeat(1, len(GridSearch_metrics)), 'b': np.repeat(1, len(GridSearch_metrics)),
                           'completion_time':np.repeat(1, len(GridSearch_metrics))})

GridSearch_metrics = pd.concat([GridSearch_metrics, key_values], axis = 1)

GridSearch_metrics

,alpha,n_iter,n_trials,MSE,R^2,a,b,completion_time,MSE,R^2,a,b,completion_time
0,0.001,100,5,1,1,1,1,1,1,1,1,1,1
1,0.001,100,10,1,1,1,1,1,1,1,1,1,1
2,0.001,1000,5,1,1,1,1,1,1,1,1,1,1
3,0.001,1000,10,1,1,1,1,1,1,1,1,1,1
4,0.001,10000,5,1,1,1,1,1,1,1,1,1,1
5,0.001,10000,10,1,1,1,1,1,1,1,1,1,1
6,0.010,100,5,1,1,1,1,1,1,1,1,1,1
7,0.010,100,10,1,1,1,1,1,1,1,1,1,1
8,0.010,1000,5,1,1,1,1,1,1,1,1,1,1
9,0.010,1000,10,1,1,1,1,1,1,1,1,1,1


In [ ]:
for i in range(0, len(GridSearch_metrics)):

  alpha = GridSearch_metrics.iloc[i, 0]

  n_iter = GridSearch_metrics.iloc[i, 1]

  n_trials = GridSearch_metrics.iloc[i, 2]


  GridSearch_metrics.iloc[i, 3] = DIY_LinReg_metrics_for_given_HP_MEANS(alpha, n_iter, n_trials).iloc[:,0].item()

  GridSearch_metrics.iloc[i, 4] = DIY_LinReg_metrics_for_given_HP_MEANS(alpha, n_iter, n_trials).iloc[:,1].item()

  GridSearch_metrics.iloc[i, 5] = DIY_LinReg_metrics_for_given_HP_MEANS(alpha, n_iter, n_trials).iloc[:,2].item()

  GridSearch_metrics.iloc[i, 6] = DIY_LinReg_metrics_for_given_HP_MEANS(alpha, n_iter, n_trials).iloc[:,3].item()

  GridSearch_metrics.iloc[i, 7] = DIY_LinReg_metrics_for_given_HP_MEANS(alpha, n_iter, n_trials).iloc[:,4].item()








In [ ]:
#save so I don't have to rerun
GridSearch_metrics.to_csv('GS_metrics_DIY_LINREG_scaling.csv')

In [ ]:
GridSearch_metrics_scaling = pd.read_csv('GS_metrics_DIY_LINREG_scaling.csv').iloc[:, 1:9]

In [ ]:
GridSearch_metrics_scaling.sort_values(by = ['R^2'], ascending = False).head(20)



,alpha,n_iter,n_trials,MSE,R^2,a,b,completion_time
13,0.100,100,10,10.779354,0.601432,0.059312,7.012736,0.098188
23,1.000,10000,10,10.837797,0.596384,0.047528,7.017422,10.476671
20,1.000,1000,5,10.746901,0.595795,0.047981,7.115457,0.479813
21,1.000,1000,10,10.765311,0.595190,0.047803,7.025069,0.967438
16,0.100,10000,5,10.812151,0.593382,0.047487,7.031338,5.334718
5,0.001,10000,10,12.453420,0.592596,0.047576,7.028745,10.584205
15,0.100,1000,10,10.780753,0.592580,0.047571,7.031442,0.950847
17,0.100,10000,10,10.799037,0.592570,0.047524,7.031366,10.909661
11,0.010,10000,10,10.794737,0.592558,0.047566,7.028419,10.538445
10,0.010,10000,5,10.793928,0.592532,0.047570,7.030457,5.247527


In [ ]:
#conclusions

# Up to row with index 22, the MSE and R^2 values are comparable to sklearn
#But after row with index 4, the performance is much worse. Having alpha of 1 or 0.1
#increases likelihood of good peformance

#In terms of speed vs accuracy, setting n_iter to 10000 doesn't improve
#accuracy much but significantly worsens speed

#n_trials = 10 doesn't improve accuracy as much compared to n_trials = 5

#one conclusion from all of this is that version 2 performs well with the right parameters

#that said, a big problem is that we do need the right parameters in order to
#get a good performance in terms of speed and accuracy

#It may be that having found best parameters for this dataset, I can use same parameters for
#other datasets, but it may also be that optimum parameters vary given the data
#(i.e. due to how well the data fits a linear model)

#next I will investigate this using a different dataset



# Testing second version on new dataset

In [ ]:
#upload new data set - comparing heights and weights (treat weight as the dependent variable)

#scale data, set up test/train splits, and get benchmark performance from sklearn

#apply adapted functions from my second version of the model to compare performance

In [ ]:
hw = pd.read_csv('SOCR-HeightWeight.csv')

In [ ]:
hw.head()

,Index,Height(Inches),Weight(Pounds)
0,1,65.78331,112.9925
1,2,71.51521,136.4873
2,3,69.39874,153.0269
3,4,68.21660,142.3354
4,5,67.78781,144.2971


In [ ]:
#df.rename(columns={"A": "a", "B": "c"})

hw = hw.rename(columns = {"Height(Inches)":"H", "Weight(Pounds)":"W" })
hw.head()

,Index,H,W
0,1,65.78331,112.9925
1,2,71.51521,136.4873
2,3,69.39874,153.0269
3,4,68.21660,142.3354
4,5,67.78781,144.2971


In [ ]:
hw.isna().mean()

Index    0.0
H        0.0
W        0.0
dtype: float64

In [ ]:
len(hw)

25000

In [ ]:
test_1_index_HW = np.arange(0,5000)
test_2_index_HW = np.arange(5000,10000)
test_3_index_HW = np.arange(10000,15000)
test_4_index_HW = np.arange(15000,20000)
test_5_index_HW = np.arange(20000,25000)

def train_index_HW(test_index):

  full_index_set = set(np.arange(0,25000))
  test_index_set = set(test_index)

  train_index_set = full_index_set - test_index_set

  train_index_list = list(train_index_set)

  train_index_array = np.array(train_index_list)

  return train_index_array



train_1_index_HW = train_index_HW(test_1_index_HW)
train_2_index_HW = train_index_HW(test_2_index_HW)
train_3_index_HW = train_index_HW(test_3_index_HW)
train_4_index_HW = train_index_HW(test_4_index_HW)
train_5_index_HW = train_index_HW(test_5_index_HW)

In [ ]:
test_1_HW = hw.iloc[test_1_index_HW]
test_2_HW = hw.iloc[test_2_index_HW]
test_3_HW = hw.iloc[test_3_index_HW]
test_4_HW = hw.iloc[test_4_index_HW]
test_5_HW = hw.iloc[test_5_index_HW]

test_sets_HW = [test_1_HW, test_2_HW, test_3_HW, test_4_HW, test_5_HW]


train_1_HW = hw.iloc[train_1_index_HW]
train_2_HW = hw.iloc[train_2_index_HW]
train_3_HW = hw.iloc[train_3_index_HW]
train_4_HW = hw.iloc[train_4_index_HW]
train_5_HW = hw.iloc[train_5_index_HW]

train_sets_HW = [train_1_HW, train_2_HW, train_3_HW, train_4_HW, train_5_HW]

In [ ]:
#set up SK learn benchmarks

storage = pd.DataFrame({'Run':[], 'MSE':[], 'R^2': [], 'a': [], 'b':[], 'completion_time':[]})

for i in range(0, 5):

  start = time.time()

  X_train_mean = train_sets_HW[0].H.mean()
  X_train_std = train_sets_HW[0].H.std()

  X_train_US = train_sets_HW[i].H
  X_test_US = test_sets_HW[i].H

  X_train_Scaled = np.array((X_train_US - X_train_mean)/X_train_std).reshape(-1, 1)
  X_test_Scaled = np.array((X_test_US - X_train_mean)/X_train_std).reshape(-1, 1)

  y_train = train_sets_HW[i].W
  y_test = test_sets_HW[i].W

  model = LinearRegression()
  model.fit(X_train_Scaled,y_train)

  predictions = model.predict(X_test_Scaled)

  a_s = model.coef_.item()
  b_s = model.intercept_.item()

  a = a_s/X_train_std

  b = b_s - ((a_s*X_train_mean)/(X_train_std))

  r2 = model.score(X_test_Scaled, y_test)
  mse = mean_squared_error(y_test, predictions, squared = True)


  storage_row = pd.DataFrame({'Run':[i+1], 'MSE':[round(mse,3)], 'R^2': [round(r2,3)], 'a': [round(a,3)], 'b':[round(b,3)], 'completion_time':[time.time()- start]})

  storage = pd.concat([storage, storage_row], axis = 0)


storage = storage.reset_index()


storage_means = pd.DataFrame({'Run':['means'], 'MSE':[storage.MSE.mean()], 'R^2': [storage['R^2'].mean()], 'a': [storage.a.mean()], 'b':[storage.b.mean()], 'completion_time':[storage['completion_time'].mean()]})


storage_stds = pd.DataFrame({'Run':['stds'], 'MSE':[storage.MSE.std()], 'R^2': [storage['R^2'].std()], 'a': [storage.a.std()], 'b':[storage.b.std()], 'completion_time':[storage['completion_time'].std()]})


storage = pd.concat([storage, storage_means, storage_stds], axis = 0)

storage_sklearn_bm_HW = storage.drop(['index'], axis = 1)

storage_sklearn_bm_HW

,Run,MSE,R^2,a,b,completion_time
0,1.0,97.620000,0.267000,3.081000,-82.490000,0.005565
1,2.0,103.071000,0.252000,3.074000,-81.979000,0.003641
2,3.0,101.151000,0.252000,3.088000,-82.830000,0.004088
3,4.0,105.570000,0.234000,3.103000,-83.902000,0.002811
4,5.0,100.626000,0.259000,3.071000,-81.691000,0.002727
0,means,101.607600,0.252800,3.083400,-82.578400,0.003767
0,stds,2.954451,0.012194,0.012779,0.861654,0.001156


In [ ]:
print (storage_sklearn_bm_HW)

None


In [ ]:
#lower R^2 and higher MSE indicates there is a less strong relationship between ht and wt, relative to TV budget and Sales


In [ ]:
#set up function that I will use to test how my model performs on ht wt data
#when using the hyperparameters that worked well previously

In [ ]:
def DIY_LinReg_metrics_for_given_HP_hw(alpha, n_iter, n_trials):

  storage = pd.DataFrame({'MSE':[], 'R^2': [], 'a': [], 'b':[], 'completion_time':[]})

  for i in range(0, 5):

    storage_row = a_b_mse_r_squared_WITH_SCALING(train_sets_HW[i]['H'], train_sets_HW[i]['W'] , test_sets_HW[i]['H'], test_sets_HW[i]['W'], alpha, n_iter, n_trials)

    storage = pd.concat([storage, storage_row], axis = 0)


  storage_means = pd.DataFrame({'MSE':[storage['MSE'].mean()], 'R^2': [storage['R^2'].mean()],
                                'a': [storage.a.mean()], 'b':[storage.b.mean()], 'completion_time':[storage.completion_time.mean()]})

  storage_stds = pd.DataFrame({'MSE':[storage['MSE'].std()], 'R^2': [storage['R^2'].std()],
                                'a': [storage.a.std()], 'b':[storage.b.std()], 'completion_time':[storage.completion_time.std()]})


  storage = pd.concat([storage, storage_means, storage_stds], axis = 0)

  storage = storage.set_index([pd.Index([1, 2, 3, 4, 5, 'means', 'stds'])])

  return storage

In [ ]:
#check function works, using hyperparameters that should yield good performance

In [ ]:
DIY_LinReg_metrics_for_given_HP_hw(1, 1000, 5)

,MSE,R^2,a,b,completion_time
1,97.656490,0.267068,3.081589,-82.565349,4.539432
2,103.213254,0.250570,3.082598,-82.818535,2.453434
3,101.268859,0.251167,3.109197,-84.064570,2.423265
4,106.054785,0.230296,3.126500,-84.922873,3.975106
5,100.708342,0.258288,3.084618,-82.454087,2.504143
means,101.780346,0.251478,3.096900,-83.365083,3.179076
stds,3.112116,0.013593,0.020107,1.082150,1.004686


In [ ]:
#set up GridSearch process to find best HPs

In [ ]:
GridSearch_metrics = pd.DataFrame([(alpha, n_iter, n_trials) for alpha in [0.01, 0.1, 1] for n_iter in [100,1000] for n_trials in [5,10]],
                                  columns = ['alpha', 'n_iter', 'n_trials'])


key_values = pd.DataFrame({'MSE': np.repeat(1, len(GridSearch_metrics)), 'R^2': np.repeat(1, len(GridSearch_metrics)),
                           'a': np.repeat(1, len(GridSearch_metrics)), 'b': np.repeat(1, len(GridSearch_metrics)),
                           'completion_time':np.repeat(1, len(GridSearch_metrics))})

GridSearch_metrics = pd.concat([GridSearch_metrics, key_values], axis = 1)



In [ ]:
GridSearch_metrics.head(3)

,alpha,n_iter,n_trials,MSE,R^2,a,b,completion_time
0,0.01,100,5,1,1,1,1,1
1,0.01,100,10,1,1,1,1,1
2,0.01,1000,5,1,1,1,1,1


In [ ]:
for i in range(0, len(GridSearch_metrics)):

  alpha = GridSearch_metrics.iloc[i, 0]

  n_iter = GridSearch_metrics.iloc[i, 1]

  n_trials = GridSearch_metrics.iloc[i, 2]

#mse
  GridSearch_metrics.iloc[i, 3] = DIY_LinReg_metrics_for_given_HP_hw(alpha, n_iter, n_trials).loc['means','MSE'].item()

#r^2
  GridSearch_metrics.iloc[i, 4] = DIY_LinReg_metrics_for_given_HP_hw(alpha, n_iter, n_trials).loc['means','R^2'].item()

#a
  GridSearch_metrics.iloc[i, 5] = DIY_LinReg_metrics_for_given_HP_hw(alpha, n_iter, n_trials).loc['means','a'].item()

#b
  GridSearch_metrics.iloc[i, 6] = DIY_LinReg_metrics_for_given_HP_hw(alpha, n_iter, n_trials).loc['means','b'].item()

#time
  GridSearch_metrics.iloc[i, 7] = DIY_LinReg_metrics_for_given_HP_hw(alpha, n_iter, n_trials).loc['means', 'completion_time'].item()

In [ ]:
GridSearch_metrics.to_csv('hw_GSM.csv')

In [ ]:
GridSearch_metrics

,alpha,n_iter,n_trials,MSE,R^2,a,b,completion_time
0,0.01,100,5,12943.359677,-93.421551,2.326976,142.425479,0.253538
1,0.01,100,10,12626.174892,-87.506359,0.747619,-12.249921,0.508015
2,0.01,1000,5,10846.611659,-83.864025,3.249953,-147.157673,2.838186
3,0.01,1000,10,10576.865533,-73.817302,-4.136058,-0.150934,5.851661
4,0.10,100,5,10409.484334,-72.502068,3.341079,-264.602805,0.253046
5,0.10,100,10,10367.852389,-72.389014,3.813743,-352.620154,0.509170
6,0.10,1000,5,330.455015,-1.761762,2.518560,-54.105986,2.947522
7,0.10,1000,10,262.413504,-1.012998,3.199048,-83.005259,5.574980
8,1.00,100,5,258.656800,-0.931675,2.553232,-50.303066,0.256838
9,1.00,100,10,324.736787,-1.137442,2.902925,-79.956305,0.508887


In [ ]:
#performs well when alpha = 1, n_iter = 1000; n_trials = 5 optomizes for speed
#for all other HPs, performance is very poor

In [ ]:
#This suggests that while may model may have some safe bet parameters
#it is not guaranteed to perform well for all parameters within a reasonable range
#as such it is not going to be as reliable as sklearn in practice

In [ ]:
#At the heart of the issue is that my model is not converging on optimum solutions fast
#enough for it to be efficient in terms of speed

#I hence want to develop a DIY model that uses a technique called momentum

# Building third version of my linear regression function - Gradient Descent with Momentum

Momentum works on the principle that the change (V) made at any point (W),
should not only depend upon the gradient at W (G(W)) but also the gradients at the previous points that the algoriithm has iterated through (with decreasing significance based on how my iterations ago the algorithm got to that point).

This means that if the gradient continually changes direction with respect to one parameter, but stays the same for another (e.g. the direction of greatest increase flips from + to - for a, but stays + for b), then the change in b will get greater in the intended direction, while pertubations in a will be dampended.

The logic of momentum is that:

alpha = alpha
beta = beta

W = np.array([a_start, b_start])
G = loss_function_derivative_wrt_ab_UNIT(x,y, W[0], W[1])
V = G

for i in range(n_iter):

  W = W - V

  G = loss_function_derivative_wrt_ab_UNIT(x,y, W[0], W[1])

  V = (beta*V) + G

W = np.array([a_estimate, b_estimate])


To create an algorithm that uses momentum, I will:

- use existing loss function and gradient function
- update find_best_ab_for_given_start ...to include momentum process
- update find_best_ab ... to include updated find_best_ab_for_given_start
- a_b_mse_r_squared_WITH_SCALING accordingly


The hope is that this algorithm will converge on best a,b without
need for as many iterations or trials - and thus be faster



In [ ]:
def find_best_ab_for_given_start_MOMENTUM (X,y, a_start, b_start, alpha, beta, n_iter):

 point = np.array([a_start, b_start])
 alpha_grad = alpha * loss_function_derivative_wrt_ab_UNIT(X,y,a_start,b_start)
 V = alpha_grad

 for i in range(0, n_iter):

   point = point - V

   alpha_grad = alpha * loss_function_derivative_wrt_ab_UNIT(X,y,point[0],point[1])

   V = (beta*V) + alpha_grad

 a_est = point[0]
 b_est = point[1]


 return[a_est, b_est]

In [ ]:
def find_best_ab_MOMENTUM(X,y, alpha, beta, n_iter, n_trials):

 a_starts = np.random.randint(-20,20, n_trials)
 b_starts = np.random.randint(-20,20, n_trials)

 a_estimates = []
 b_estimates = []

 for i in range(n_trials):

   point_estimates = find_best_ab_for_given_start_MOMENTUM (X,y, a_starts[i], b_starts[i], alpha, beta, n_iter)

   a_estimates.append(point_estimates[0])
   b_estimates.append(point_estimates[1])

 SSR_values = []

 for i in range(0,n_trials):

   SSR_values.append(loss_function_uniV_LR (X, y, a_estimates[i], b_estimates[i]))

 SSR_values_min = min(SSR_values)
 min_index = SSR_values.index(SSR_values_min)

 a_min = a_estimates[min_index]
 b_min = b_estimates[min_index]


 return [a_min, b_min]



In [ ]:

def a_b_mse_r_squared_WITH_SCALING_MOMENTUM (x_train, y_train, x_test, y_test, alpha, beta, n_iter, n_trials):

  start = time.time()

  x_train_mean = np.mean(x_train)
  x_train_std = np.std(x_train)

  x_train_scaled = (x_train - x_train_mean)/x_train_std
  x_test_scaled = (x_test - x_train_mean)/x_train_std

  best_a_b = find_best_ab_MOMENTUM(x_train_scaled,y_train, alpha, beta, n_iter, n_trials)

  a_s = best_a_b[0]
  b_s = best_a_b[1]

  predictions = (a_s*x_test_scaled) + b_s

  errors = predictions - y_test

  MSE = (np.dot(errors, errors))/len(y_test)

  y_test_mean = np.mean(y_test)

  y_test_mean_vector = np.repeat(y_test_mean, len(y_test))

  errors_from_mean = y_test - y_test_mean_vector

  R_squared = 1 - ((np.dot(errors, errors))/(np.dot(errors_from_mean, errors_from_mean)))

  a = a_s/x_train_std

  b = b_s - ((a_s*x_train_mean)/(x_train_std))

  end = time.time()

  storage = pd.DataFrame({'MSE':[MSE], 'R^2': [R_squared], 'a': [a], 'b':[b], 'completion_time':[end - start]})

  return storage

In [ ]:
storage = pd.DataFrame({'MSE':[], 'R^2': [], 'a': [], 'b':[], 'completion_time':[]})

for i in range(0, 5):

  storage_row = a_b_mse_r_squared_WITH_SCALING_MOMENTUM(train_sets[i].TV, train_sets[i].Sales , test_sets[i].TV, test_sets[i].Sales, 1, 0.9, 100, 5)

  storage = pd.concat([storage, storage_row], axis = 0)

storage

,MSE,R^2,a,b,completion_time
0,10.926686,0.577263,0.038648,7.556115,0.109001
0,9.400467,0.680790,0.048361,6.430417,0.067883
0,9.623155,0.570998,0.049006,6.845936,0.071904
0,10.495218,0.641090,0.046729,7.264827,0.065849
0,13.312700,0.501471,0.052255,5.915783,0.064349


In [ ]:
def DIY_LinReg_metrics_for_given_HP_MMT(alpha, beta, n_iter, n_trials):

  storage = pd.DataFrame({'MSE':[], 'R^2': [], 'a': [], 'b':[], 'completion_time':[]})

  for i in range(0, 5):

    storage_row = a_b_mse_r_squared_WITH_SCALING_MOMENTUM(train_sets[i].TV, train_sets[i].Sales , test_sets[i].TV, test_sets[i].Sales, alpha, beta, n_iter, n_trials)

    storage = pd.concat([storage, storage_row], axis = 0)

  return storage

In [ ]:
def DIY_LinReg_metrics_for_given_HP_MEANS_MMT(alpha, beta, n_iter, n_trials):

  full_cv_df = DIY_LinReg_metrics_for_given_HP_MMT(alpha, beta, n_iter, n_trials)

  storage = pd.DataFrame({'MSE':[full_cv_df['MSE'].mean()], 'R^2': [full_cv_df['R^2'].mean()],
                          'a': [full_cv_df['a'].mean()], 'b':[full_cv_df['b'].mean()],
                          'completion_time':[full_cv_df['completion_time'].mean()]})

  return storage

In [ ]:
GridSearch_metrics = pd.DataFrame([(alpha, beta, n_iter, n_trials) for alpha in [0.01, 0.1, 1] for beta in [0.5, 0.7, 0.9] for n_iter in [100,500,1000] for n_trials in [5,10,15]],
                                 columns = ['alpha', 'beta','n_iter', 'n_trials'])


key_values = pd.DataFrame({'MSE': np.repeat(1, len(GridSearch_metrics)), 'R^2': np.repeat(1, len(GridSearch_metrics)),
                          'a': np.repeat(1, len(GridSearch_metrics)), 'b': np.repeat(1, len(GridSearch_metrics)),
                          'completion_time':np.repeat(1, len(GridSearch_metrics))})


GridSearch_metrics_MMT_TV = pd.concat([GridSearch_metrics, key_values], axis = 1)


GridSearch_metrics_MMT_TV


,alpha,beta,n_iter,n_trials,MSE,R^2,a,b,completion_time
0,0.01,0.5,100,5,1,1,1,1,1
1,0.01,0.5,100,10,1,1,1,1,1
2,0.01,0.5,100,15,1,1,1,1,1
3,0.01,0.5,500,5,1,1,1,1,1
4,0.01,0.5,500,10,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
76,1.00,0.9,500,10,1,1,1,1,1
77,1.00,0.9,500,15,1,1,1,1,1
78,1.00,0.9,1000,5,1,1,1,1,1
79,1.00,0.9,1000,10,1,1,1,1,1


In [ ]:
for i in range(0, len(GridSearch_metrics_MMT_TV)):

  alpha = GridSearch_metrics_MMT_TV.iloc[i, 0]
  beta = GridSearch_metrics_MMT_TV.iloc[i,1]
  n_iter = GridSearch_metrics_MMT_TV.iloc[i, 2]
  n_trials = GridSearch_metrics_MMT_TV.iloc[i, 3]

  GridSearch_metrics_MMT_TV.iloc[i, 4] = DIY_LinReg_metrics_for_given_HP_MEANS_MMT(alpha, beta, n_iter, n_trials).iloc[:,0].item()
  GridSearch_metrics_MMT_TV.iloc[i, 5] = DIY_LinReg_metrics_for_given_HP_MEANS_MMT(alpha, beta, n_iter, n_trials).iloc[:,1].item()
  GridSearch_metrics_MMT_TV.iloc[i, 6] = DIY_LinReg_metrics_for_given_HP_MEANS_MMT(alpha, beta, n_iter, n_trials).iloc[:,2].item()
  GridSearch_metrics_MMT_TV.iloc[i, 7] = DIY_LinReg_metrics_for_given_HP_MEANS_MMT(alpha, beta, n_iter, n_trials).iloc[:,3].item()
  GridSearch_metrics_MMT_TV.iloc[i, 8] = DIY_LinReg_metrics_for_given_HP_MEANS_MMT(alpha, beta, n_iter, n_trials).iloc[:,4].item()

In [ ]:
pd.set_option('display.max_rows', None)
GridSearch_metrics_MMT_TV.sort_values('R^2', ascending = False)

,alpha,beta,n_iter,n_trials,MSE,R^2,a,b,completion_time
54,1.00,0.5,100,5,11.228358,0.598470,0.048142,7.038445,0.034219
57,1.00,0.5,500,5,10.763867,0.597499,0.048549,7.131402,0.164427
67,1.00,0.7,500,10,10.786794,0.595496,0.047663,7.097880,0.338758
75,1.00,0.9,500,5,11.732942,0.595240,0.046010,6.720033,0.164539
45,0.10,0.9,100,5,10.796042,0.594714,0.047363,7.013841,0.034983
62,1.00,0.5,1000,15,10.822485,0.593252,0.048110,7.022438,1.152012
43,0.10,0.7,1000,10,10.806395,0.593178,0.047659,7.029617,0.665501
55,1.00,0.5,100,10,10.800642,0.593037,0.047269,6.999221,0.072925
37,0.10,0.7,100,10,10.808369,0.593032,0.047525,7.028682,0.067400
58,1.00,0.5,500,10,10.826871,0.592944,0.047614,6.941815,0.335846


In [ ]:
GridSearch_metrics_MMT_TV.to_csv('GridSearch_metrics_MMT_TV.csv')

In [ ]:
#conclusions:

#alpha of 0.01 is not effective, alpha of 1 performs slightly better than 0.1 on the whole
#n_iter of 1000 is unnessacary, can perform well with 100 or 500
#not sensitive to beta when beta in range of 0.5, 0.9
#n_trials of 5 works as well as 10


#want to see how beta outside of current range perform
#want to see if n_trials can be as low as 1/2/3
#want to see how alpha performs if greater than 1 - to check overal senstivity

In [ ]:
DIY_LinReg_metrics_for_given_HP_MEANS_MMT(1, 0.1, 500, 5)

,MSE,R^2,a,b,completion_time
0,11.009172,0.584826,0.047402,7.035677,0.249266


In [ ]:
DIY_LinReg_metrics_for_given_HP_MEANS_MMT(1, 0.2, 500, 5)

,MSE,R^2,a,b,completion_time
0,10.958547,0.586533,0.047596,7.105699,0.474115


In [ ]:
DIY_LinReg_metrics_for_given_HP_MEANS_MMT(1, 0.3, 500, 5)

,MSE,R^2,a,b,completion_time
0,10.731795,0.594696,0.047971,6.996728,0.360461


In [ ]:
#beta still performs well below 0.5-0.9 range

In [ ]:
#want to see how it performs close to 0 and 1

In [ ]:
DIY_LinReg_metrics_for_given_HP_MEANS_MMT(1, 0.01, 500, 5)

,MSE,R^2,a,b,completion_time
0,10.900336,0.588649,0.046971,7.178214,0.375689


In [ ]:
DIY_LinReg_metrics_for_given_HP_MEANS_MMT(1, 0.99, 500, 5)

,MSE,R^2,a,b,completion_time
0,11.221694,0.573909,0.045331,6.878948,0.347051


In [ ]:
#conclusion, for alpha = 1, n_iter = 500 - model is not that sensitive to beta

In [ ]:
DIY_LinReg_metrics_for_given_HP_MEANS_MMT(1, 0.01, 100, 5)

,MSE,R^2,a,b,completion_time
0,10.77286,0.593237,0.047031,7.045903,0.036324


In [ ]:
DIY_LinReg_metrics_for_given_HP_MEANS_MMT(1, 0.99, 100, 5)

,MSE,R^2,a,b,completion_time
0,20.673393,0.241989,0.043807,8.792735,0.039159


In [ ]:
#seems beta cannot be too close to one for lower number of iterations, and higher alpha

In [ ]:
#can n-trials be less than 5?

In [ ]:
DIY_LinReg_metrics_for_given_HP_MEANS_MMT(1, 0.5, 500, 5)

,MSE,R^2,a,b,completion_time
0,10.796566,0.592814,0.046474,7.040351,0.173679


In [ ]:
DIY_LinReg_metrics_for_given_HP_MEANS_MMT(1, 0.5, 500, 1)

,MSE,R^2,a,b,completion_time
0,10.733295,0.594289,0.047796,7.403136,0.069411


In [ ]:
DIY_LinReg_metrics_for_given_HP_MEANS_MMT(1, 0.5, 500, 2)

,MSE,R^2,a,b,completion_time
0,10.813108,0.591957,0.046126,7.295617,0.162347


In [ ]:
DIY_LinReg_metrics_for_given_HP_MEANS_MMT(1, 0.5, 500, 3)

,MSE,R^2,a,b,completion_time
0,10.859409,0.589447,0.048037,6.904428,0.203458


In [ ]:
DIY_LinReg_metrics_for_given_HP_MEANS_MMT(1, 0.5, 500, 4)

,MSE,R^2,a,b,completion_time
0,10.666533,0.597209,0.047567,7.18249,0.329921


In [ ]:
#conclusion: there does seem to be an advantage to n_iter = 5

#How do larger alphas affect performance?

In [ ]:
DIY_LinReg_metrics_for_given_HP_MEANS_MMT(2, 0.5, 500, 5)

,MSE,R^2,a,b,completion_time
0,11.22147,0.577795,0.048528,6.722595,0.266616


In [ ]:
DIY_LinReg_metrics_for_given_HP_MEANS_MMT(5, 0.5, 500, 4)

,MSE,R^2,a,b,completion_time
0,12.226356,0.535296,0.052334,5.947512,0.290173


In [ ]:
#seems that there is some sensitivity to larger values

# Testing third version on new dataset

In [ ]:
#benchmark against
storage_sklearn_bm_HW

,Run,MSE,R^2,a,b,completion_time
0,1.0,97.620000,0.267000,3.081000,-82.490000,0.005565
1,2.0,103.071000,0.252000,3.074000,-81.979000,0.003641
2,3.0,101.151000,0.252000,3.088000,-82.830000,0.004088
3,4.0,105.570000,0.234000,3.103000,-83.902000,0.002811
4,5.0,100.626000,0.259000,3.071000,-81.691000,0.002727
0,means,101.607600,0.252800,3.083400,-82.578400,0.003767
0,stds,2.954451,0.012194,0.012779,0.861654,0.001156


In [ ]:
storage_sklearn_bm_HW.to_csv('storage_sklearn_bm_HW.csv')

In [ ]:
def DIY_LinReg_metrics_for_given_HP_hw_MMT(alpha, beta, n_iter, n_trials):

  storage = pd.DataFrame({'MSE':[], 'R^2': [], 'a': [], 'b':[], 'completion_time':[]})

  for i in range(0, 5):

    storage_row = a_b_mse_r_squared_WITH_SCALING_MOMENTUM(train_sets_HW[i]['H'], train_sets_HW[i]['W'] , test_sets_HW[i]['H'], test_sets_HW[i]['W'], alpha, beta, n_iter, n_trials)

    storage = pd.concat([storage, storage_row], axis = 0)


  storage_means = pd.DataFrame({'MSE':[storage['MSE'].mean()], 'R^2': [storage['R^2'].mean()],
                                'a': [storage.a.mean()], 'b':[storage.b.mean()], 'completion_time':[storage.completion_time.mean()]})

  storage_stds = pd.DataFrame({'MSE':[storage['MSE'].std()], 'R^2': [storage['R^2'].std()],
                                'a': [storage.a.std()], 'b':[storage.b.std()], 'completion_time':[storage.completion_time.std()]})


  storage = pd.concat([storage, storage_means, storage_stds], axis = 0)

  storage = storage.set_index([pd.Index([1, 2, 3, 4, 5, 'means', 'stds'])])

  return storage

In [ ]:
DIY_LinReg_metrics_for_given_HP_hw_MMT(1, 0.7, 500, 5)

,MSE,R^2,a,b,completion_time
1,97.580613,0.267638,3.080364,-82.369021,1.027261
2,103.698117,0.247049,3.022047,-79.085819,2.017427
3,101.134931,0.252157,3.076436,-82.224416,1.049822
4,105.560629,0.233882,3.100581,-83.735820,0.952322
5,100.713469,0.258250,3.082143,-82.279800,0.953392
means,101.737552,0.251795,3.072314,-81.938975,1.200045
stds,3.047612,0.012611,0.029598,1.714040,0.459004


In [ ]:
DIY_LinReg_metrics_for_given_HP_hw_MMT(1, 0.7, 100, 5)

,MSE,R^2,a,b,completion_time
1,97.539773,0.267944,3.081098,-82.332599,0.626546
2,103.071293,0.251600,3.074380,-81.982963,0.561104
3,101.154176,0.252015,3.087524,-82.812970,0.493175
4,105.568804,0.233823,3.103631,-83.920903,0.544528
5,100.590896,0.259153,3.075161,-82.114596,0.555126
means,101.584988,0.252907,3.084359,-82.632806,0.556096
stds,2.984019,0.012568,0.012003,0.786227,0.047650


In [ ]:
#MMT model performs well for second dataset, and it did not require lengthy Hyperparameter tuning to achieve this.